In [47]:
# Some cleanup
%reset
# For SciDB temp arrays
import os
os.system('''iquery -aq "remove(foo)"''')
os.system('''iquery -aq "remove(foo)"''')
os.system('''iquery -aq "remove(foo3)"''')

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


256

First create the target array

In [48]:
os.system('''iquery -aq "create array foo <density:uint8> [frequency=0:199,200,0,time=0:*,1,0]"''')

0

Next, let us create an array from numpy as given in the query

In [49]:
import numpy as np
x = np.arange(50).reshape(10,5)
x

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49]])

In [50]:
import scidbpy as sdb
sdb = sdb.connect()

temp = sdb.from_array(x)
temp.schema

u'<f0:int64 NOT NULL> [i0=0:9,1000,0,i1=0:4,1000,0]'

Note that `x` did not have any information about the time axis. So that information is not to be expected in the result of `from_array`.

Anyway, let us try to insert the array `temp` into `foo`

In [51]:
#sdb.afl.insert(temp, 'foo') 
# You can uncomment and try
# This gives an error: "Error during schema inferring. Arrays are not conformant: Source array for INSERT/STORE must have uniform chunk sizes."

We have to invoke the `redimension` operator to convert the source array `temp` to have the same dimensions as `foo`

In [52]:
#sdb.afl.redimension(temp2, '<density:uint8> [frequency=0:199,200,0,time=0:*,1,0]') 
# You can uncomment and try
# This gives the error: "Error during schema inferring. No data provided for destination attribute 'density'."

This is because original `foo` array has the attribute as `uint8` but numpy array has type `double` -- so there is a type mismatch. Let's resolve this. 

In [53]:
temp2=sdb.afl.apply(temp, 'density', 'uint8(f0)')
temp2.name

u'apply(py1467055182482052038_00011,density,uint8(f0))'

Now let us apply the `redimension` after this change (note `f0` will be dropped)

In [54]:
temp3 = sdb.afl.redimension(temp2, '<density:uint8 NOT NULL> [i0=0:*,200,0,i1=0:*,1,0]')
temp3.name

u'redimension(apply(py1467055182482052038_00011,density,uint8(f0)),<density:uint8 NOT NULL> [i0=0:*,200,0,i1=0:*,1,0])'

and then try to run an `insert`

In [55]:
temp4 = sdb.afl.insert(temp3, 'foo')
temp4.name

u'insert(redimension(apply(py1467055182482052038_00011,density,uint8(f0)),<density:uint8 NOT NULL> [i0=0:*,200,0,i1=0:*,1,0]),foo)'

Finally let us evaluate the expression

In [56]:
temp4.eval()

SciDBArray()

Now let us see the current state of the `foo` array

In [57]:
sdb.wrap_array('foo').todataframe().head()

,,density
frequency,time,
0,0,0
1,0,5
2,0,10
3,0,15
4,0,20


Note that the `time` axis still does not start at `time=50:54` as required in the original query. 

But as noted before, that info never existed in the original numpy array in the first place. So that info has to be added. Two options exist:

## Option 1

One option is to add that information post upload of numpy array to SciDB. This example is shown below -- note use of `apply` operator, and that `redimension` now drops both `f0` and `i1`.

In [58]:
os.system('''iquery -aq "create array foo2 <density:uint8> [frequency=0:199,200,0,time=0:*,1,0]"''')
temp2 = sdb.afl.apply(temp, 'time', 'i1+50', \
                      'density', 'uint8(f0)')
temp3 = sdb.afl.redimension(temp2, '<density:uint8 NOT NULL> [i0=0:*,200,0,time=0:*,1,0]')
temp4 = sdb.afl.insert(temp3, 'foo2')
temp4.eval()
sdb.wrap_array('foo2').todataframe().head()

,,density
frequency,time,
0,50,0
1,50,5
2,50,10
3,50,15
4,50,20


## Option 2
1. Contain the `time` value within another numpy array that has the same structure as the data numpy array, join the arrays after loading into SciDB-Py and then redimension to only the required dimensions

In [59]:
x

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49]])

In [60]:
np.arange(50, 55) * np.ones(50).astype(int).reshape(10,5)

array([[50, 51, 52, 53, 54],
       [50, 51, 52, 53, 54],
       [50, 51, 52, 53, 54],
       [50, 51, 52, 53, 54],
       [50, 51, 52, 53, 54],
       [50, 51, 52, 53, 54],
       [50, 51, 52, 53, 54],
       [50, 51, 52, 53, 54],
       [50, 51, 52, 53, 54],
       [50, 51, 52, 53, 54]])

In [61]:
timearr = sdb.from_array(np.arange(50, 55) * np.ones(50).astype(int).reshape(10,5))
timearr.schema

u'<f0:int64 NOT NULL> [i0=0:9,1000,0,i1=0:4,1000,0]'

In [62]:
temp1=sdb.afl.apply(temp, 'density', 'uint8(f0)')
temp2 = sdb.merge(temp1, timearr)
temp2.eval()
temp2.schema

u'<f0:int64 NOT NULL,density:uint8 NOT NULL,f0_2:int64 NOT NULL> [i0=0:9,1000,0,i1=0:4,1000,0]'

In [63]:
os.system('''iquery -aq "create array foo3 <density:uint8> [frequency=0:199,200,0,time=0:*,1,0]"''')
temp3 = sdb.afl.redimension(temp2, '<density:uint8 NOT NULL> [i0=0:*,200,0,f0_2=0:*,1,0]') #Both `f0` and `i1` are dropped
temp4 = sdb.afl.insert(temp3, 'foo3')
temp4.eval()
sdb.wrap_array('foo3').todataframe().head()

,,density
frequency,time,
0,50,0
1,50,5
2,50,10
3,50,15
4,50,20


In [64]:
# Some cleanup
os.system('''iquery -aq "remove(foo)"''')
os.system('''iquery -aq "remove(foo2)"''')
os.system('''iquery -aq "remove(foo3)"''')

0